In [0]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.optimizers import Adam , rmsprop

Using TensorFlow backend.


### Loading & splitting data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pandas.read_csv("boston_data.csv",  header=None, skiprows=1,sep=',')

In [0]:
dataset = df.values
X = dataset[:,0:13]
Y = dataset[:,-1]

### Creating baseline model 

In [0]:
def baseline_model():
    model=Sequential()
    model.add(Dense(13, activation='relu' ,input_shape=(13,)))
    model.add(Dense(1))
    model.compile(Adam(lr=0.002),loss='mean_squared_error')
    return model

In [0]:
seed = 7
numpy.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [0]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -38.13 (19.71) MSE


### Standardize dataset

In [0]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Standardized: -0.01 (0.01) MSE


### Rescaling the output layer with activation function in baseline model

In [0]:
def rescaled_model():
    model=Sequential()
    model.add(Dense(13, activation='relu' ,input_shape=(13,)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(Adam(lr=0.001),loss='mean_squared_error')
    return model

In [0]:
seed = 7
numpy.random.seed(seed)
estimator = KerasRegressor(build_fn=rescaled_model, epochs=50, batch_size=5, verbose=0)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=rescaled_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Rescaled_Model_Result: %.2f (%.2f) MSE" % (results.mean(), results.std()))


ValueError: ignored

### Rescaling the baseline model 1

In [0]:
def larger_model():
    model=Sequential()
    model.add(Dense(13, activation='relu' ,input_shape=(13,)))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))
    model.compile(Adam(lr=0.001),loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

In [0]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Larger: -22.63 (28.30) MSE


### Rescaling the baseline model 2

In [0]:
def wider_model():
    model=Sequential()
    model.add(Dense(20, activation='relu' ,input_shape=(13,)))
    model.add(Dense(1))
    model.compile(Adam(lr=0.002),loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

In [0]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -22.41 (21.21) MSE


### Creating overfitting model

In [0]:
def overfitting_model():
    model=Sequential()
    model.add(Dense(20, activation='relu' ,input_shape=(13,)))
    model.add(Dense(13, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1))
    model.compile(Adam(lr=0.08),loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

In [0]:
seed = 7
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=overfitting_model, epochs=200, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -67.74 (52.15) MSE


### Retunning the model 

In [0]:
def retuning_model():
    model=Sequential()
    model.add(Dense(39, activation='relu' ,input_shape=(13,)))
    model.add(Dense(13, activation='relu'))
    model.add(Dense(1))
    model.compile(Adam(lr=0.002),loss='mean_squared_error')
    return model

In [0]:
seed = 7
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=retuning_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=20, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Retuning_Result: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Retuning_Result: -22.31 (38.51) MSE


### Creating model with keras functional API 

In [0]:
import keras
from keras import layers

def functional_api():
    inputs=keras.Input(shape=(13,))
    hl_1=layers.Dense(13,activation='relu')(inputs)
    outputs=layers.Dense(1)(hl_1)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer=Adam(lr=0.002),loss='mean_squared_error')
    return model

In [0]:
estimator = KerasRegressor(build_fn=functional_api, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("functional_API_Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

functional_API_Result: -3357.12% (1332.49%)


###Sub-Classing Model

In [0]:
import keras
input=keras.Input(shape=(4,))
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.dense1=layers.Dense(64,activation='relu')
        self.dense2=layers.Dense(32,activation='relu')
        self.dense3=layers.Dense(16,activation='relu')
        self.dense4=layers.Dense(3,activation='softmax')
    def call(self,inputs):
        x=self.dense1(inputs)
        x=self.dense2(x)
        x=self.dense3(x)
        return self.dense4(x)
model=MyModel()
model.compile(optimizer=Adam(lr=0.002),loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X,Y,epochs=50,batch_size=3)

Epoch 1/50
506/506 [==============================] - 1s 2ms/step - loss: 127.5442 - acc: 0.0000e+00
Epoch 2/50
506/506 [==============================] - 0s 476us/step - loss: 127.6829 - acc: 0.0000e+00
Epoch 3/50
506/506 [==============================] - 0s 461us/step - loss: 127.6959 - acc: 0.0000e+00
Epoch 4/50
506/506 [==============================] - 0s 463us/step - loss: 127.6959 - acc: 0.0000e+00
Epoch 5/50
506/506 [==============================] - 0s 467us/step - loss: 127.6959 - acc: 0.0000e+00
Epoch 6/50
506/506 [==============================] - 0s 461us/step - loss: 127.6959 - acc: 0.0000e+00
Epoch 7/50
506/506 [==============================] - 0s 473us/step - loss: 127.6959 - acc: 0.0000e+00
Epoch 8/50
506/506 [==============================] - 0s 459us/step - loss: 127.6959 - acc: 0.0000e+00
Epoch 9/50
506/506 [==============================] - 0s 467us/step - loss: 127.6959 - acc: 0.0000e+00
Epoch 10/50
506/506 [==============================] - 0s 465us/step - loss

### Applying K-Fold Cross validation without Scikit learn

In [0]:
def baseline_model2():
  #create model
  input = keras.Input(shape=(13,))
  a = layers.Dense(13,activation='relu')(input)
  a = layers.Dense(3,activation='relu')(a)
  outputs = layers.Dense(1)(a)
  #compile model
  model = keras.Model(input, outputs)
  model.compile(optimizer='Adam',loss='mse',metrics=['accuracy'])
  model.fit(X,Y,epochs=20,batch_size=32,verbose=0)
  return model

In [0]:
k = 4
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []

In [30]:
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = Y[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = numpy.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = numpy.concatenate([Y[:i * num_val_samples],Y[(i + 1) * num_val_samples:]],axis=0)
model = baseline_model2()
model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
all_scores.append(val_mae)
score = numpy.average(all_scores)


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [31]:

print("Result: %.2f%% (%.2f%%)" % (score.mean()*100,score.std()*100))

Result: 3652.84% (0.00%)
